# 남자 배구 전처리

## 원본 파일 DF화 및 기초 전처리

In [1]:
# Import module 
import pandas as pd

In [2]:
# 남자 배구 csv 데이터 파일 읽기
df_male = pd.read_csv('./male_volleyball.csv')

In [3]:
# 팀명 변경 : OK저축은행' -> 'OK금융그룹
df_male['팀명'] = df_male['팀명'].replace({'OK저축은행':'OK금융그룹'})

# 경기 날짜 datetime으로 타입 변경
df_male['경기날짜'] = pd.to_datetime(df_male['경기날짜'])

## 피쳐(경기력) 전처리

In [4]:
# 필요한 컬럼 추출
df_male_mod = df_male[['팀명', '경기날짜', '결과', '득점_득점',
    '공격종합_시도', '공격종합_성공',
    '후위_시도', '후위_성공',
    '퀵오픈_시도', '퀵오픈_성공',
    '서브_시도', '서브_성공',
    '디그_시도', '디그_성공',
    '세트_시도', '세트_성공',
    '리시브_시도', '리시브_정확',
    '블로킹_시도', '블로킹_성공']]

# 팀명, 경기날짜 순으로 합계 그룹화
df_male_mod = df_male_mod.groupby(['팀명', '경기날짜']).sum()

# 결과 칼럼 값 변경 : 승리 - 1, 패배 - 0
df_male_mod['결과'] = df_male_mod['결과'].apply(lambda x: 1 if x != 0 else 0)

# 멀티인덱스 해제
df_male_mod = df_male_mod.reset_index()

# 각 조건별 성공률 컬럼 생성
df_male_mod['공격종합성공률'] = df_male_mod['공격종합_성공'] / df_male_mod['공격종합_시도']
df_male_mod.drop(columns=['공격종합_성공', '공격종합_시도'], inplace=True)

df_male_mod['후위성공률'] = df_male_mod['후위_성공'] / df_male_mod['후위_시도']
df_male_mod.drop(columns=['후위_성공', '후위_시도'], inplace=True)

df_male_mod['퀵오픈성공률'] = df_male_mod['퀵오픈_성공'] / df_male_mod['퀵오픈_시도']
df_male_mod.drop(columns=['퀵오픈_성공', '퀵오픈_시도'], inplace=True)

df_male_mod['서브성공률'] = df_male_mod['서브_성공'] / df_male_mod['서브_시도']
df_male_mod.drop(columns=['서브_성공', '서브_시도'], inplace=True)

df_male_mod['디그성공률'] = df_male_mod['디그_성공'] / df_male_mod['디그_시도']
df_male_mod.drop(columns=['디그_시도', '디그_성공'], inplace=True)

df_male_mod['세트성공률'] = df_male_mod['세트_성공'] / df_male_mod['세트_시도']
df_male_mod.drop(columns=['세트_성공', '세트_시도'], inplace=True)

df_male_mod['리시브성공률'] = df_male_mod['리시브_정확'] / df_male_mod['리시브_시도']
df_male_mod.drop(columns=['리시브_정확', '리시브_시도'], inplace=True)

df_male_mod['블로킹성공률'] = df_male_mod['블로킹_성공'] / df_male_mod['블로킹_시도']
df_male_mod.drop(columns=['블로킹_성공', '블로킹_시도'], inplace=True)

## 결측치 처리

In [5]:
# 결측치 제거
df_male_mod.fillna(0, inplace=True)

## 피쳐(상대전적) 전처리

In [6]:
# 상대 전적 만들기
df_male_vs = df_male[['팀명','경기날짜','결과']]

# 중복 데이터 제거 (중복되는 첫 번째 행은 남김)
df_male_vs = df_male_vs.drop_duplicates(keep = 'first')
df_male_vs

# 팀 이름, 경기날짜 리스트 생성
team_list = df_male_vs['팀명'].unique()
date_list = df_male_vs['경기날짜'].unique()

# 조건 1 승리한 팀
cond1 = df_male_vs['결과'] == 1

# 조건 2 날짜별
for date in date_list:
    cond2 = df_male_vs['경기날짜'] == date
    
    # 상대팀 이름 변수에 저장
    team1 = df_male_vs[cond1 & cond2]['팀명'].values[0]    # 승리팀
    team2 = df_male_vs[~cond1 & cond2]['팀명'].values[0]   # 패배팀
    
    # '상대팀' 열에 상대팀 명 저장
    df_male_vs.loc[cond1 & cond2,'상대팀'] = team2 # 승리팀의 상대팀 정보 저장
    df_male_vs.loc[~cond1 & cond2,'상대팀'] = team1 # 패배팀의 상대팀 정보 저장

## 경기력과 상대전적 merge & One-Hot Encoding

In [7]:
# 병합
df_male_mod = pd.merge(df_male_mod, df_male_vs)

# 팀명 원핫인코딩
df_male_mod = pd.get_dummies(df_male_mod , columns=['팀명', '상대팀'])

## 17.10 ~ 22.02 데이터와 22.03 ~ 04 데이터 분리

In [8]:
# 2월 train 데이터와 3-4월 test 데이터 나누기
year = df_male_mod['경기날짜'].dt.year
month = df_male_mod['경기날짜'].dt.month
cond_date = (year == 2022) & (month > 2)

train_male = df_male_mod[~cond_date]
test_male = df_male_mod[cond_date]

# 경기 날짜 칼럼 제거
train_male.drop(columns='경기날짜', inplace=True)
test_male.drop(columns='경기날짜', inplace=True)

C:\Users\admin\AppData\Local\Temp\ipykernel_6640\1555831880.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_male.drop(columns='경기날짜', inplace=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_6640\1555831880.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_male.drop(columns='경기날짜', inplace=True)


## CSV 파일 저장

In [9]:
# csv 파일로 저장
train_male.to_csv('train_male.csv')
test_male.to_csv('test_male.csv')